In [1]:
import pandas as pd
import numpy as np

In [2]:
ttc = pd.read_csv('ttc_delays_cleaned.csv')

In [3]:
ttc_stations = ttc['Station']
ttc_stations.head(2)

0        HIGH PARK
1    SHEPPARD - YU
Name: Station, dtype: object

In [4]:
stations = pd.read_csv('csv_originals/station_ridership_grade.csv')
stations['Station'] = stations['Station'].str.upper()
stations.head(2)

,Station,Grade,2015 ridership
0,COLLEGE,Underground,"47,790"
1,DAVISVILLE,Surface,"25,330"


In [5]:
station_index = stations.set_index('Station')
station_index.index

Index(['COLLEGE', 'DAVISVILLE', 'DUNDAS', 'DUPONT', 'EGLINTON',
       'EGLINTON WEST', 'FINCH', 'GLENCAIRN', 'KING', 'LAWRENCE',
       'LAWRENCE WEST', 'MUSEUM', 'NORTH YORK CENTRE', 'OSGOODE', 'QUEEN',
       'QUEEN'S PARK', 'ROSEDALE', 'SHEPPARD WEST', 'ST. ANDREW', 'ST. CLAIR',
       'ST. CLAIR WEST', 'ST. PATRICK', 'SUMMERHILL', 'UNION', 'WELLESLEY',
       'WILSON', 'YORK MILLS', 'YORKDALE', 'BLOOR-YONGE - BD',
       'BLOOR-YONGE - YU', 'SPADINA - YU', 'SPADINA - BD', 'ST. GEORGE - YU',
       'ST. GEORGE - BD', 'SHEPPARD - SHP', 'SHEPPARD - YU', 'BATHURST', 'BAY',
       'BROADVIEW', 'CASTLE FRANK', 'CHESTER', 'CHRISTIE', 'COXWELL',
       'DONLANDS', 'DUFFERIN', 'DUNDAS WEST', 'GREENWOOD', 'HIGH PARK',
       'ISLINGTON', 'JANE', 'KEELE', 'KIPLING', 'LANSDOWNE', 'MAIN STREET',
       'OLD MILL', 'OSSINGTON', 'PAPE', 'ROYAL YORK', 'RUNNYMEDE',
       'SHERBOURNE', 'VICTORIA PARK', 'WARDEN', 'WOODBINE', 'KENNEDY - SRT',
       'KENNEDY - BD', 'ELLESMERE', 'LAWRENCE EAST', 'MCC

In [6]:
ttc_stations_index = pd.Index(ttc_stations.unique())
ttc_stations_index

Index([         'HIGH PARK',      'SHEPPARD - YU',          'LANSDOWNE',
         'BLOOR-YONGE - YU',           'DUFFERIN',  'NORTH YORK CENTRE',
                'RUNNYMEDE',              'QUEEN',         'ST. ANDREW',
                'WELLESLEY',            'KIPLING',        'ST. PATRICK',
             'KENNEDY - BD',               'JANE',           'OLD MILL',
                   'WILSON',          'DOWNSVIEW',                'BAY',
                    'FINCH',            'COXWELL',     'ST. CLAIR WEST',
                   'DUPONT',         'YORK MILLS',        'DUNDAS WEST',
                  'OSGOODE',            'COLLEGE',   'BLOOR-YONGE - BD',
            'VICTORIA PARK',    'ST. GEORGE - YU',     'SHEPPARD - SHP',
                'BROADVIEW',           'EGLINTON',           'ROSEDALE',
                     'PAPE',          'GREENWOOD',          'ISLINGTON',
                   'WARDEN',         'DAVISVILLE',           'LAWRENCE',
            'EGLINTON WEST',              'BLOOR', 

Lets find which of the Stations don't match

In [7]:
station_index.index.difference(ttc_stations_index)

Index(['MAIN STREET', 'SHEPPARD WEST'], dtype='object')

In [8]:
ttc_stations_index.difference(station_index.index)

Index([         nan,      'BLOOR',  'DOWNSVIEW',    'KENNEDY',       'MAIN',
         'SHEPPARD',    'SPADINA', 'ST. GEORGE',      'YONGE'],
      dtype='object')

Looks like we need to fix:
BLOOR–YONGE --> Bloor and Yonge?
MAIN STREET --> MAIN
SHEPPARD WEST --> SHEPPARD
SHEPPARD–YONGE --> DOWNSVIE

In [10]:
# Remove the STREET from main street like we did in the other file
stations.loc[stations['Station'] == 'MAIN STREET'] = 'MAIN'

# Rename Sheppard West to DOWNSVIEW because this data predates 2017
stations.loc[stations['Station'] == 'SHEPPARD WEST'] = 'DOWNSVIEW'

In [11]:
station_index = stations.set_index('Station')
ttc_stations_index = pd.Index(ttc_stations.unique())
station_index.index.difference(ttc_stations_index)

Index([], dtype='object')

In [12]:
ttc_stations_index.difference(station_index.index)

Index([nan, 'BLOOR', 'KENNEDY', 'SHEPPARD', 'SPADINA', 'ST. GEORGE', 'YONGE'], dtype='object')

In [13]:
ttc_exchange_stations = ttc[ttc['Station'].isin(['BLOOR', 'SHEPPARD', 'YONGE'])]

In [14]:
ttc_exchange_stations[['Station', 'Line']].drop_duplicates().sort_values('Station')

,Station,Line
69,BLOOR,NaN
15880,SHEPPARD,NaN
20421,SHEPPARD,Bloor-Danforth
1178,YONGE,NaN


In [15]:
ttc_exchange_stations.groupby(['Station', 'Line']).size()

Station   Line          
SHEPPARD  Bloor-Danforth    37
dtype: int64